In [1]:
#word segmenter
import py_vncorenlp
import os
    
#py_vncorenlp.download_model(save_dir='/VnCoreNLP')

here = os.path.dirname(os.path.abspath('main.ipynb'))
os.chdir(here)

rdrsegmenter = py_vncorenlp.VnCoreNLP(save_dir=os.path.join(here, 'VnCoreNLP'))

In [2]:
import numpy
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModel
from keras.preprocessing.sequence import pad_sequences
import torch

# Load pre-trained PhoBERT tokenizer and model
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")

# Define a function to tokenize and extract features from text
def extract_features(text):
    #process text
    text = rdrsegmenter.word_segment(text)
    text = " ".join(text)
    ids = phobert_tokenizer.encode(text)
    ids_padded = pad_sequences([ids], maxlen=256, dtype="long", value=0, truncating="post", padding="post")
    ids_padded = ids_padded[0]
    mask = [int(token_id > 0) for token_id in ids_padded]
    ids_input = torch.tensor(ids_padded).to(torch.long).reshape(1,-1)
    input_mask = torch.tensor(mask).reshape(1,-1)
    
    #ids_input = ids_input.squeeze(0)
    
    with torch.no_grad():
        features = phobert_model(input_ids=ids_input, attention_mask=input_mask)
    
    return features[0][:, 0, :].numpy()

# Load and preprocess your labeled dataset
# You'll need to prepare your own labeled dataset with texts and corresponding difficulty labels.
# Here, we assume you have a list of examples in the format (text, label).
def ReadData():
    data = []

    # Define the labels
    labels = ["difficult", "medium", "easy", "veryeasy"]

    root_directory = here
    dataset_directory = root_directory + "/data"

    # Iterate over the labels
    for label in labels:
        # Iterate over the files in the folder
        for filename in os.listdir(dataset_directory):
            # Check if the file matches the label
            if filename.startswith(label):
                # Read the contents of the file
                with open(os.path.join(dataset_directory, filename), "r", encoding="utf-8") as file:
                    text = file.read()
                # Append the text and label to the data list
                data.append((text, label))

    return data

def train():
    
    data = ReadData()
    
    X = []
    y = []

    for text, label in data:
        features = extract_features(text)
        X.append(features[0])
        y.append(label)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a classifier (e.g., Random Forest)
    classifier = SVC(kernel='linear', probability=True, gamma=0.125)
    classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")
    
    return classifier
   
def predict(text, model):
    features = extract_features(text)
    prediction = model.predict(features[0])
    return prediction    
    
#main    

#train model
svc_model = train()

#using pickle to save or load model



c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token indices sequence length is longer than the specified maximum sequence length for this model (6422 > 256). Running this sequence through the model will result in indexing errors


Accuracy: 0.7643835616438356


In [5]:
import pickle


# save
with open(here + '/model.pkl','wb') as f:
    pickle.dump(svc_model,f)

# load
#with open('model_pkl', 'rb') as f:
#    svc_model = pickle.load(f)
